# AG2 - Ramificación y Poda: Asignación de tareas
*Adrien Felipe*    
https://colab.research.google.com/drive/1GVrR1zB1qq-sGhDVFBVCtFFSvecdwfsd

In [0]:
import itertools
import random
from time import time

## Decorador tiempo de ejecución

In [0]:
def execution_time(function):
    """Function decorator to track execution time, in milliseconds"""
    def wrapper(*args, **kwargs):
        start_time = time()       
        result = function(*args, **kwargs)       
        print("Execution time: %i ms" % int((time() - start_time) * 1000))
        
        return result
    
    return wrapper

## Código clase

In [0]:
# Problema de asignación de tarea
COSTES=[
  [11,12,18,40],
  [14,15,13,22],
  [11,17,19,23],
  [17,14,20,28],
]

In [0]:
# Coste de una solución
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i]
  return VALOR

In [0]:
# Coste inferior
def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(  len(COSTES)  ):
    if i not in S:
      VALOR += min( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

In [0]:
# Coste superior
def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]
  
  #Estimacion
  for i in range(  len(COSTES)   ):
    if i not in S:
      VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

In [0]:
# Genera tantos hijos como posibilidades haya para el siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N ):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)    })
  return HIJOS

In [0]:
@execution_time
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(COSTES)
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  #print("Cota Superior:", CotaSup)

  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((),COSTES)    } )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
    if len(NODO_FINAL ) >0: 
      #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION  ] )
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
  print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )

In [10]:
ramificacion_y_poda(COSTES)

La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  14  iteraciones  para dimension:  4
Execution time: 3 ms


## Fuerza bruta

In [0]:
@execution_time
def fuerza_bruta(COSTES):
  mejor_valor = 10e10
  mejor_solucion = ()

  for s in list(itertools.permutations(range(len(COSTES)))):
    valor_tmp = valor(s, COSTES)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s

  print('La mejor solución es: %s con valor: %s' % (mejor_solucion, mejor_valor))

In [12]:
fuerza_bruta(COSTES)

La mejor solución es: (0, 3, 1, 2) con valor: 61
Execution time: 0 ms


## Preguntas

### ¿Que complejidad tiene el algoritmo por fuerza bruta?
La complejidad es factorial O(n!)

### Generar matrices con valores aleatorios de mayores dimensiones

In [13]:
dims = [5,6,7,10,11]
COSTES_EXTRA = {str(dim): [[int(random.random() * 50) for i in range(dim)] for j in range(dim)] for dim in dims}

for dim, LOCAL_COSTES in COSTES_EXTRA.items():
  print('\n\n# Dim: %s -----------------\n' % dim)
  print('Ramificación y poda:')
  ramificacion_y_poda(LOCAL_COSTES)
  print()
  print('Fuerza bruta:')
  fuerza_bruta(LOCAL_COSTES)



# Dim: 5 -----------------

Ramificación y poda:
La solucion final es: [{'s': (1, 4, 0, 3, 2), 'ci': 77}]  en  18  iteraciones  para dimension:  5
Execution time: 2 ms

Fuerza bruta:
La mejor solución es: (2, 0, 4, 3, 1) con valor: 77
Execution time: 0 ms


# Dim: 6 -----------------

Ramificación y poda:
La solucion final es: [{'s': (4, 2, 1, 0, 5, 3), 'ci': 55}]  en  24  iteraciones  para dimension:  6
Execution time: 2 ms

Fuerza bruta:
La mejor solución es: (3, 2, 1, 5, 0, 4) con valor: 55
Execution time: 1 ms


# Dim: 7 -----------------

Ramificación y poda:
La solucion final es: [{'s': (0, 4, 2, 3, 6, 5, 1), 'ci': 48}]  en  13  iteraciones  para dimension:  7
Execution time: 1 ms

Fuerza bruta:
La mejor solución es: (0, 6, 2, 3, 1, 5, 4) con valor: 48
Execution time: 6 ms


# Dim: 10 -----------------

Ramificación y poda:
La solucion final es: [{'s': (5, 2, 7, 9, 0, 4, 6, 8, 3, 1), 'ci': 86}]  en  679  iteraciones  para dimension:  10
Execution time: 72 ms

Fuerza bruta:
La m

### ¿A partir de que dimensión el algoritmo por fuerza bruta deja de ser una opción?
A partir de 10 dimensiones, ya se dispara el tiempo de ejecución del algoritmo por fuerza bruta, mientras que el de ramificación sigue tardando fracciones de segundo.    
De hecho, mas de 11 dimensiones colab se queda sin RAM.

### ¿Hay algún valor de la dimensión a partir de la cual el algoritmo de ramificación y poda deja de ser una opción válida?

In [18]:
dims = range(11,16)
COSTES_EXTRA = {str(dim): [[int(random.random() * 50) for i in range(dim)] for j in range(dim)] for dim in dims}

for dim, LOCAL_COSTES in COSTES_EXTRA.items():
  print('# Dim: %s -----------------' % dim)
  print('Ramificación y poda:')
  ramificacion_y_poda(LOCAL_COSTES)
  print()

# Dim: 11 -----------------
Ramificación y poda:
La solucion final es: [{'s': (5, 9, 4, 3, 6, 8, 10, 0, 1, 7, 2), 'ci': 74}]  en  1073  iteraciones  para dimension:  11
Execution time: 199 ms

# Dim: 12 -----------------
Ramificación y poda:
La solucion final es: [{'s': (4, 1, 8, 0, 5, 7, 3, 10, 9, 11, 6, 2), 'ci': 54}]  en  1027  iteraciones  para dimension:  12
Execution time: 163 ms

# Dim: 13 -----------------
Ramificación y poda:
La solucion final es: [{'s': (2, 6, 12, 8, 5, 3, 7, 11, 0, 1, 10, 9, 4), 'ci': 49}]  en  8248  iteraciones  para dimension:  13
Execution time: 8393 ms

# Dim: 14 -----------------
Ramificación y poda:
La solucion final es: [{'s': (1, 4, 2, 12, 6, 8, 0, 10, 3, 13, 9, 11, 7, 5), 'ci': 48}]  en  831  iteraciones  para dimension:  14
Execution time: 202 ms

# Dim: 15 -----------------
Ramificación y poda:
La solucion final es: [{'s': (6, 3, 2, 10, 5, 8, 0, 14, 12, 13, 11, 9, 4, 1, 7), 'ci': 89}]  en  13809  iteraciones  para dimension:  15
Execution time: 24

Vemos que a partir de **dimensión 15**, el tiempo de ejecución del algoritmo de ramificación y poda se dispara. Posiblemente no se pueda superar la dimensión 20 en tiempos racionales.